<a href="https://colab.research.google.com/github/Zain1796/Lucky-Cement-AI-System/blob/main/ner_peipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.is_available()

True

In [ ]:
pip install torchaudio librosa transformers openpyxl pandas torch

In [ ]:
import os
import torchaudio
import librosa
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
import torch
import json

In [ ]:
model, utils = torch.hub.load('snakers4/silero-vad', model='silero_vad', force_reload=True)
transcriber = pipeline(model="/content/drive/MyDrive/Joyn_NewDataNLP/models/whisper-feb-24-HASH_V1.0.33", task="automatic-speech-recognition", chunk_length_s=30, device=0)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Joyn_NewDataNLP/models/NER_finetuned_latest_checkpoint_5000/checkpoint-5000")
model_ner = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/Joyn_NewDataNLP/models/NER_finetuned_latest_checkpoint_5000/checkpoint-5000")
nlp = pipeline("ner", model=model_ner, tokenizer=tokenizer, device=0)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
map_label = {
    'O': 0, 'DG_Landed': 1, 'DG_Retail': 2, 'DG_Margin': 3, 'DG_Sale Quantity': 4, 'DG_Stock': 5, 'DG_Supplier': 6,
    'Lucky_Landed': 7, 'Lucky_Retail': 8, 'Lucky_Margin': 9, 'Lucky_Sale Quantity': 10, 'Lucky_Stock': 11, 'Lucky_Supplier': 12,
    'Thatta_Landed': 13, 'Thatta_Retail': 14, 'Thatta_Margin': 15, 'Thatta_Sale Quantity': 16, 'Thatta_Stock': 17, 'Thatta_Supplier': 18,
    'PakLand_Landed': 19, 'PakLand_Retail': 20, 'PakLand_Margin': 21, 'PakLand_Sale Quantity': 22, 'PakLand_Stock': 23, 'PakLand_Supplier': 24,
    'Power_Landed': 25, 'Power_Retail': 26, 'Power_Margin': 27, 'Power_Sale Quantity': 28, 'Power_Stock': 29, 'Power_Supplier': 30,
    'Falcon_Landed': 31, 'Falcon_Retail': 32, 'Falcon_Margin': 33, 'Falcon_Sale Quantity': 34, 'Falcon_Stock': 35, 'Falcon_Supplier': 36,
    'Other_Landed': 37, 'Other_Retail': 38, 'Other_Margin': 39, 'Other_Sale Quantity': 40, 'Other_Stock': 41, 'Other_Supplier': 42,
    'Payment_Method': 43
}

In [ ]:
# Directory containing audio files
audio_dir = "/content/drive/MyDrive/Joyn_NewDataNLP/Audio_RawData/Aug_05_2024"
output_excel = "/content/drive/MyDrive/Joyn_NewDataNLP/Excel_sheets/ner_infreances/Aug_05_2024_comparison.xlx"
# writer = pd.ExcelWriter(output_excel, engine='openpyxl')

In [ ]:
# Function to convert MP3 to WAV
def convert_mp3_to_wav(mp3_file, output_file):
    waveform, sample_rate = torchaudio.load(mp3_file)
    torchaudio.save(output_file, waveform, sample_rate=sample_rate)
    return output_file

# Function for Voice Activity Detection (VAD)
def vad(audio_file, model, utils):
    (get_speech_timestamps, save_audio, read_audio, _, collect_chunks) = utils
    wav = read_audio(audio_file, sampling_rate=16000)
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=16000)

    # Check if any speech is detected
    if not speech_timestamps:
        raise RuntimeError("No speech detected in the audio file.")

    vad_output_path = f"only_speech_{os.path.basename(audio_file)}"
    save_audio(vad_output_path, collect_chunks(speech_timestamps, wav), sampling_rate=16000)
    return vad_output_path

# Function to split text into chunks for NER
def split_text_into_chunks(text, max_length=512, overlap=50):
    words = text.split()
    chunks, current_chunk, current_length = [], [], 0
    for word in words:
        word_tokens = tokenizer.tokenize(word)
        if current_length + len(word_tokens) > max_length - 2:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:]
            current_length = sum([len(tokenizer.tokenize(w)) for w in current_chunk])
        current_chunk.append(word)
        current_length += len(word_tokens)
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# Function for NER inference on text chunks
def ner_inference_on_chunks(chunks):
    results = []
    for chunk in chunks:
        results.extend(nlp(chunk))
    return results

# Function to combine chunk results into continuous entities
def combine_chunk_results(chunk_results):
    combined_results = []
    for result in chunk_results:
        if len(combined_results) == 0 or result['start'] >= combined_results[-1]['end']:
            combined_results.append(result)
        else:
            combined_results[-1]['end'] = max(combined_results[-1]['end'], result['end'])
            combined_results[-1]['word'] += result['word']
    return combined_results

# Function to map entities to keys
def map_entity_to_key(entity):
    number = entity.split('_')[-1]
    for key, value in map_label.items():
        if value == int(number):
            return key
    return None  # Return None if not found

# Function to process NER results and format them
def process_ner_results(final_results):
    combined_data = combine_chunk_results(final_results)
    filtered_data = [item for item in combined_data if item['entity'] != 'LABEL_0']

    # Create a DataFrame with only 'word' and 'entity' columns
    df = pd.DataFrame(filtered_data, columns=['entity', 'word'])
    df['mapped_key'] = df['entity'].apply(map_entity_to_key)
    df.drop(columns=['entity'], inplace=True)
    print(df)
    # Prepare the result DataFrame
    expected_companies = ['DG', 'Lucky', 'Thatta', 'PakLand', 'Power', 'Falcon', 'Other']
    expected_attributes = ['Landed', 'Retail', 'Margin', 'Sale Quantity', 'Stock', 'Supplier']

    result_df = pd.DataFrame(index=expected_attributes, columns=expected_companies)
    result_df = result_df.fillna('None')
    check_payment = 'None'
    for _, row in df.iterrows():
        if row['mapped_key'] == "Payment_Method":
          check_payment = row['word']
          continue
        mapped_key_split = row['mapped_key'].split('_')
        company = mapped_key_split[0]
        attribute = mapped_key_split[1]
        value = row['word']

        if company in expected_companies and attribute in expected_attributes:
            if result_df.at[attribute, company] == 'None':
                result_df.at[attribute, company] = value
            else:
                result_df.at[attribute, company] += value

    print(result_df)
    result_dict = result_df.to_dict()
    result_dict["Payment_Method"] = check_payment
    return result_dict


# Process audio file
def audio_to_json(filename):
    out_json = {}
    # for filename in os.listdir(audio_dir)[6:7]:
    print(filename)
    out_json["id"] = filename
    if filename.endswith(".mp3"):
        mp3_file_path =  filename
        wav_output_file = f"{os.path.splitext(filename)[0]}.wav"

      # Step 1: Convert MP3 to WAV
        wav_file_path = convert_mp3_to_wav(mp3_file_path, wav_output_file)

        try:
          # Step 2: Voice Activity Detection (VAD)
            vad_output_path = vad(wav_file_path, model, utils)
        except RuntimeError as e:
            print(f"Skipping {filename} due to VAD error: {e}")

      # Step 3: Transcription
        speech_audio, _ = librosa.load(vad_output_path, sr=16000)
        transcription = transcriber(speech_audio)["text"]
        out_json["transcription"] = transcription
        print(transcription)

        # Step 4: Named Entity Recognition (NER)
        chunks = split_text_into_chunks(transcription)
        print(chunks)
        chunk_results = ner_inference_on_chunks(chunks)
        final_results = combine_chunk_results(chunk_results)

        # Step 5: Process NER results into a formatted DataFrame
        result_dict = process_ner_results(final_results)
        out_json["ner_results"] = result_dict

    return out_json

In [ ]:
import os
import json

# Assuming hlp.audio_to_json function exists and processes the audio into a JSON object

def process_audios_to_json(folder_path):
    # List all audio files in the folder
    audio_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3')]

    for audio_file in audio_files:
        # Construct the full path to the audio file
        audio_path = os.path.join(folder_path, audio_file)

        # Call the function to convert the audio to JSON
        audio_json = audio_to_json(audio_path)

        # Create the JSON file path with the same name as the audio file but with .json extension
        json_file_path = os.path.join('outputs', audio_file.replace('.mp3', '.json'))

        # Save the JSON data to the file
        with open(json_file_path, 'w') as json_file:
            json.dump(audio_json, json_file, ensure_ascii=False, indent=4)

# Use the function
process_audios_to_json('/content/test_audios_lcl')

/content/test_audios_lcl/Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.mp3


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


بس یہ بھی تیار ہے ہاں یہ پہلی چال ہے وہ دوسری چال ہے چلو صحیح ہے # ہاں پر لکھا لیکن # لیکن آج اس سیمنٹ کوئی بندے ہیں کافی سر پے ہیں ابھی نہ کسر آپ ہیں پہلے کوئی اور آتا تا ہے # پاک لینڈ بیلنے اس کے لئے چلو 5 1020  بوری تمہارا ہی نہیں # وہ لوگ چلا دیا ہے کیا # اچھا ابھی آپ پاور کر رہے ہیں ویسے # پاور اتا ہے # پاور کیا ریٹ آرہا ہوگا آپ کے پاسے # پاور او پی سی پے انسے آرہا ہے 850 # 1150 کونچ # او پی سی # تو آپ اپنی گاڑی بھی جو آتے ہیں گودام میں کرایا محنس ہوتا ہوں ڈیلر کون ہے # ڈیلر تو دومن کیلر # تو اس کے ساتھ آپ کے پیمنٹ وغیرہ کیش چلتی ہے کے کتنا ہے # 20  روپے آیا تھا 30  روپے آیا تھا # کام کے حساب سے احمد ریگاوری تیز ہوتی ہے تو بیلںس کم ہوجاتا ہوگا  # بیلس کم ہوجاتا ہوگا # بس یہ ایسے تو ٹائم سے ہیں نہ کچھ بیلس کم ہوجاتے ہیں بیلسوں میں کچھ کہتے نہیں ہیں # لکی کے بارے میں کچھ سوچیں گے  # لکی کو دے دیں تو نہیں سکتا بھی یہ کسی پرویجیکٹ پر کام آئیگا کہتے ہیں یار لکی صحیح ہے پروجیکٹ کا مگر کمالا ہوتا ہے تو وہ ہے نا # ڈائریکٹ کمپنی لینڈڈے تک # اچھا اچھا پھر آپ تو مطلب مندے دیں میں وہاں سے بھی

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


کیا مسلہ ہوگا # دوسروں کو ہمارے کو نہیں مل رہا ہے  ابھی بلیک بل بھی اتارا ہے # بلیک بل بھی صبح ماشااللہ اتارا ہے # بلیک بل کے شریف بھائی سے # ادھار پے روکڑا چاہیے # ادھار پے بھی ریٹ تو کچھ ہوگا نا یا ایسے ادھار پے بس 1060 1060 کا پاک لینڈ # 1020  کا پاک لینڈ # 1010  کا ٹھٹھہ # یہ کلو طاہ ہے سمیرا وہ رکھی آج مرے کلوے مکنے ہے # اچھا تو بچی نہ #
['کیا مسلہ ہوگا # دوسروں کو ہمارے کو نہیں مل رہا ہے ابھی بلیک بل بھی اتارا ہے # بلیک بل بھی صبح ماشااللہ اتارا ہے # بلیک بل کے شریف بھائی سے # ادھار پے روکڑا چاہیے # ادھار پے بھی ریٹ تو کچھ ہوگا نا یا ایسے ادھار پے بس 1060 1060 کا پاک لینڈ # 1020 کا پاک لینڈ # 1010 کا ٹھٹھہ # یہ کلو طاہ ہے سمیرا وہ رکھی آج مرے کلوے مکنے ہے # اچھا تو بچی نہ #']
Empty DataFrame
Columns: [word, mapped_key]
Index: []
                 DG Lucky Thatta PakLand Power Falcon Other
Landed         None  None   None    None  None   None  None
Retail         None  None   None    None  None   None  None
Margin         None  None   None    None  None   None  None
Sale Quantity  

...

کیسے ہو سجاد بھائی خیریت سے ہو# نہا، اچھا بیٹھو چاہیے نہی # خیریت لیا میں نے بس رکا تھا فون یوز کرنے کے لئے # مال لیا ہے # ہاں وہ بندہ صحیح چلو # یار اسکا پتا ہے #  یہ نکلا اس کے اوپر میں نے ورنے اس نے پتا بے اوکو کی کیا کی 4 پیگ نکلے اس نے اٹھا کے باہر باہر پتھر نہیں ہوتا پتھر اب کیا ہوتا ہے ویسا سجاد بھائی اس کے لئے کیا کیا جاتا کہ آپ لے کے جاتے ہو گاڑی آتی ہے آپ کو بورڈر اتار کے وہ پیٹ ڈال لے تھی اس نے اتار دیتے ہیں پھر اتار دیتے ہیں گودام پے یار یہ 4 پیٹ آرہے ہیں خراب ہوئے # سمجھتا ہوں وہ یوٹیلیسے کرنے کے لئے توقابلوں اس کو گرانڈر مجینے ڈالا جاتا ہے اب انہوں نے باری اس میں رکھ دیے وہ تو جن گیا ہے وہ 2 پیٹ تو ویسے پھٹ کے ہوئے خراب ہوگے تو اس کے 2 پیٹ اس نے نے جو رکھوا ہے وہ زیادہ خراب ہوگے تو # وہ نہیں ہے # میں نے تو میں پوچھا ہوں مجھے انہوں نے بتایا 15 16 دن بعد # ٹھیک ہے # اب مجھے عمران بھی کہ رہا ہے یار یہ تو مجھے اتنا لیٹ کیوں بتا رہے ہیں میں نے کہا عمران بھائی میرے کو پے کر دو میرے اپنی پارٹی ہے کیش کی پارٹی ہے # بولا چل رہی ہے # اسکو بولا یار چینج کروانے ہیں اسٹاپ کا اٹھا کے پ

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


آشاءاللہ اتارا ہاں 35 ٹن 70 ٹن # 40 ٹن آتا ہے # 40 ٹن تم نے بولا تھا 2 گاڑیاں مالا ہے # تو دو کی گاڑی آتا ہے ایک گاڑی جو ہے اگر وہ پے لیٹ اس کی جو پیمنٹ ہے نا پھر وہ پاریک کو کرنی ہے # صحیح ہے # یہ بھی اچھا پیلیفٹ دے رہا ہے زانیال کو ڈیلی کا 1 لاکھ روپے کا پرافٹ ہے # ہاں # ڈیلی کا 1 لاکھ روپے کا پرافٹ لگا دیتا ہے 1 لاکھ روپے کچھ بھی جو ہے اس کا اپنا فرشا ہے سمجھل # 3 4 تجاریوں نے بھیج دیے  تم لوگوں نے ایک کو پر اتا ہوگا مہینے کا باہر  جتنا خرچےب سب نکال # یہ تم اپنا بتا رہے ہیں # دانیال کا سیٹ لکی کا والے گا # وہ دانیال نہیں ہے وہ ساما ہے دانیال میں ہوں # ہاں ساما ہے # میں وہی کہ رہا ہوں 1 پر اسکو مہینے کا بسکتا ہے 40 50  روپے کا اپنا خرچا ہوا ہے # یار # ہے نہ یار بندے رکھے ہوے ہیں تقریبن 1 2 3 4 5 6 7 کے ساتھ بندے رکھے ہوئے ہیں آفسے ان کا ادھر  # یار اپنا ڈیلرشپ میں ہیں ڈیلرشپ میں کتنا پیسا پتا کرتا ہے # یار کروڑوں روپے تو تجھے ویسا ہی چاہیے ڈیلرشپ لینے کے لئے # کیوں ابھی نقمانے پڑے گئے # ہاں وہ ایک تمہارے سیکیورٹی کل آتا ہے پھر اس کے بعد آجاتا ہے میرے بھائی # 3 4 پر رکھو نہ پڑے گا # 

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


نہیں نہیں نہیں 2 منٹ ٹھیک ہے # نہیں نہیں نہیں # بھائی جو ہے نہ ہٹ پہ گئے ہوئے ہیں یہ ہوتا ہے یہ چھوٹا سا چوہفا لے کر آیا ہے# یہ کتنے پاس ہے # باہر سے باہر سے # داڑھی پڑے گے نہ # داڑھی بھائی # یہ کیا ریٹ لگایا آپ کو رحمت بھائی نے # وہی تو آپ کو # 1090  # نیو # اچھا # ہاں # اسے تو دے بھی رہے تھے نا پھر میں نے پیسوں کا شکایت کر لے بھائی نے یہاں کے انسو ادھار لے کے دے بھی ہے فیلکن تو یہاں پے وہ ایسی ہی آرہا ہے فیلکن کو دیے ہوں ابھی بھی تھوڑے حالات ایسے ہوں فیلکن کتنے میں آرہا ہے # فیلکن کو آرہا ہے 1 کرایا ہفتے میں 40  میں آرہا ہے کسٹمر تھے تو چاہیے تھا وہ میں نے انکوئر  دو دن پہلے ہونی کے پیسے دے دیں بعد میں یہ وہ روڑے انہوں نے روک کیا میں نے کہا میں نے دیکھا بندے کے پیسے دے کہ رہے ہیں آپ کا ڈیو بہت زیادہ ہے آپ کا مال  لے کے سب میں نے ب نے بولا جو بھی یہ ہوگا مصیب بھائی کرا دیں گے آپ کو ایکوں نہیں ہے 10 دن کی بات ہے یہ تو کرا دے بندے کا کام پوچھا اس سے پیسے کی لے گی میں نے پیسے کی لے گی آپ کو پاور کا کام ہوں گاؤں # بہت لازمی گاکلے تو لے او ہے # امر کا پریشن کھانا ہے اس نے دیا ڈیٹا ہے ادھر 

In [ ]:
# prompt: zip the outputs folder

!zip -r /content/outputs.zip /content/outputs


  adding: content/outputs/ (stored 0%)
  adding: content/outputs/Daniyal_Rauf_2_ZC4_Hafeez_Cement_Wed_Aug_21_2024_165501_GMT0500_PKT.json (deflated 71%)
  adding: content/outputs/Daniyal_Rauf_2_ZC4_KHAN_FALCON_CEMENT_DEPO_Sat_Sep_07_2024_160308_GMT0500_PKT.json (deflated 71%)
  adding: content/outputs/Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.json (deflated 70%)
  adding: content/outputs/Danish_Ahmed_ZD-5_Haji_Hussain_Mushtaq_Tue_May_30_2023_153331_GMT0500.json (deflated 75%)
  adding: content/outputs/Ahsanullah_Khan_ZE7_Amaan_Steel_Wed_Jun_26_2024_124826_GMT0500_PKT.json (deflated 72%)


In [ ]:
audio_to_json('/content/Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.mp3')

/content/Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.mp3


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


    word       mapped_key
0   ▁850     Power_Landed
1    ▁11     Power_Landed
2     50     Power_Landed
3    ▁12    Falcon_Landed
4     60    Falcon_Landed
5  ▁عالم  Falcon_Supplier
6   ▁شاہ  Falcon_Supplier
                 DG Lucky Thatta PakLand      Power     Falcon Other
Landed         None  None   None    None  ▁850▁1150      ▁1260  None
Retail         None  None   None    None       None       None  None
Margin         None  None   None    None       None       None  None
Sale Quantity  None  None   None    None       None       None  None
Stock          None  None   None    None       None       None  None
Supplier       None  None   None    None       None  ▁عالم▁شاہ  None


{'id': '/content/Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.mp3',
 'transcription': 'بس یہ بھی تیار ہے ہاں یہ پہلی چال ہے وہ دوسری چال ہے چلو صحیح ہے # ہاں پر لکھا لیکن # لیکن آج اس سیمنٹ کوئی بندے ہیں کافی سر پے ہیں ابھی نہ کسر آپ ہیں پہلے کوئی اور آتا تا ہے # پاک لینڈ بیلنے اس کے لئے چلو 5 1020  بوری تمہارا ہی نہیں # وہ لوگ چلا دیا ہے کیا # اچھا ابھی آپ پاور کر رہے ہیں ویسے # پاور اتا ہے # پاور کیا ریٹ آرہا ہوگا آپ کے پاسے # پاور او پی سی پے انسے آرہا ہے 850 # 1150 کونچ # او پی سی # تو آپ اپنی گاڑی بھی جو آتے ہیں گودام میں کرایا محنس ہوتا ہوں ڈیلر کون ہے # ڈیلر تو دومن کیلر # تو اس کے ساتھ آپ کے پیمنٹ وغیرہ کیش چلتی ہے کے کتنا ہے # 20  روپے آیا تھا 30  روپے آیا تھا # کام کے حساب سے احمد ریگاوری تیز ہوتی ہے تو بیلںس کم ہوجاتا ہوگا  # بیلس کم ہوجاتا ہوگا # بس یہ ایسے تو ٹائم سے ہیں نہ کچھ بیلس کم ہوجاتے ہیں بیلسوں میں کچھ کہتے نہیں ہیں # لکی کے بارے میں کچھ سوچیں گے  # لکی کو دے دیں تو نہیں سکتا بھی یہ کسی پرویجیکٹ پر کام آئیگا کہتے ہیں یار لکی صحیح ہے پروجیکٹ 

In [ ]:
out_json

{'id': 'Ahsanullah_Khan_ZE7_Zeenat_Block_Mon_Aug_05_2024_105556_GMT0500_PKT.mp3',
 'transcription': 'بس یہ بھی تیار ہے ہاں یہ پہلی چال ہے وہ دوسری چال ہے چلو صحیح ہے # ہاں پر لکھا لیکن # لیکن آج اس سیمنٹ کوئی بندے ہیں کافی سر پے ہیں ابھی نہ کسر آپ ہیں پہلے کوئی اور آتا تا ہے # پاک لینڈ بیلنے اس کے لئے چلو 5 1020  بوری تمہارا ہی نہیں # وہ لوگ چلا دیا ہے کیا # اچھا ابھی آپ پاور کر رہے ہیں ویسے # پاور اتا ہے # پاور کیا ریٹ آرہا ہوگا آپ کے پاسے # پاور او پی سی پے انسے آرہا ہے 850 # 1150 کونچ # او پی سی # تو آپ اپنی گاڑی بھی جو آتے ہیں گودام میں کرایا محنس ہوتا ہوں ڈیلر کون ہے # ڈیلر تو دومن کیلر # تو اس کے ساتھ آپ کے پیمنٹ وغیرہ کیش چلتی ہے کے کتنا ہے # 20  روپے آیا تھا 30  روپے آیا تھا # کام کے حساب سے احمد ریگاوری تیز ہوتی ہے تو بیلںس کم ہوجاتا ہوگا  # بیلس کم ہوجاتا ہوگا # بس یہ ایسے تو ٹائم سے ہیں نہ کچھ بیلس کم ہوجاتے ہیں بیلسوں میں کچھ کہتے نہیں ہیں # لکی کے بارے میں کچھ سوچیں گے  # لکی کو دے دیں تو نہیں سکتا بھی یہ کسی پرویجیکٹ پر کام آئیگا کہتے ہیں یار لکی صحیح ہے پروجیکٹ کا مگر کم

In [ ]:
!python --version

Python 3.10.12


In [ ]:
# prompt: https://luckycai.notebook.me-south-1.sagemaker.aws/proxy/8080/predict. make a request to this url and send {'file_path':'s3://lcl-voice-samples/Ahsanullah_ZE-7_Maqbool_Block_Sat_Jan_21_2023_114814_GMT0500.mp3'} and show me the response json

import requests
import json

url = 'https://luckycai.notebook.me-south-1.sagemaker.aws/proxy/8080/predict'
data = {'file_path':'s3://lcl-voice-samples/Ahsanullah_ZE-7_Maqbool_Block_Sat_Jan_21_2023_114814_GMT0500.mp3'}

response = requests.post(url, json=data)

if response.status_code == 200:
  result_json = response.json()
  print(result_json)
else:
  print("Error:", response.status_code)


JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
import requests
import json

url = 'https://luckycai.notebook.me-south-1.sagemaker.aws/proxy/8080/predict'
data = {'file_path':'s3://lcl-voice-samples/Ahsanullah_ZE-7_Maqbool_Block_Sat_Jan_21_2023_114814_GMT0500.mp3'}

response = requests.post(url, json=data)

# Check the response status code
if response.status_code == 200:
  # Try to decode the response as JSON, handling potential errors
  try:
    result_json = response.json()
    print(result_json)
  except json.JSONDecodeError as e:
    print("Error decoding JSON response:", e)
    print("Raw response content:", response.text) # Print the raw response for debugging
else:
  print("Error:", response.status_code)
  print("Raw response content:", response.text) # Print the raw response for debugging

Error decoding JSON response: Expecting value: line 2 column 1 (char 1)
Raw response content: 
<!DOCTYPE html>
<html lang="en">
<head><style >html { background-color: #f2f3f3; }</style></head>
<body class="">
  <script type="text/javascript">
    (function() {
        var redirectUrl = function getHashArgsRedirectUrl(location, hashArgsQueryParam, oauthStartQueryParam, oauthFlowStartTime) {
    const hashArgs = "#" + (location.href.split("#")[1] || "");
    const redirectUrl = new URL(location.href);
    redirectUrl.pathname = location.pathname
        .replace(/\\/g, "/")
        .replace(/\/\/+/g, "/");
    redirectUrl.hash = "";
    redirectUrl.searchParams.append(hashArgsQueryParam, hashArgs);
    if (oauthFlowStartTime && oauthStartQueryParam) {
        redirectUrl.searchParams.append(oauthStartQueryParam, oauthFlowStartTime.toString());
    }
    return redirectUrl;
};
        window.location.replace(redirectUrl(window.location, "hashArgs", "oauthStart", 1725309985516).toString())

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.14
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.27.10.28.29
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread             

In [ ]:
# prompt: check ubuntu version

!lsb_release -a


No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [ ]:
# prompt: get py versions

!python --version


Python 3.10.12
